In [1]:
import pymysql
import numpy as np
# Define user input mapping function
def map_user_raw_input(user_data_original):
    # 입력 데이터를 복사하여 원본을 변경하지 않음
    mapped_data = user_data_original.copy()
    # _________________________________________________________________________________________________________________
    # tobacco 매핑 (담배 개비 수)
    tobacco_amount = mapped_data.get('tobacco')
    try:
        # 개비 수를 정수로 변환. 입력이 없거나 숫자가 아니면 None으로 처리
        # np.nan 대신 None을 사용하여 MySQL의 NULL과 일치시킴
        mapped_data['tobacco'] = int(tobacco_amount) if tobacco_amount is not None and tobacco_amount != '' else None
    except (ValueError, TypeError): # Added TypeError for None
        mapped_data['tobacco'] = None # 숫자로 변환할 수 없는 경우
    # ____________________________________________________________________________________________________________________
    # BE3_31 (걷기 빈도): 설문 코드를 주당 일(day) 평균으로 매핑
    walk_freq_map = {
        1: 0,     # 사용자 값 1 -> 0일 (전혀 하지 않음)
        2: 1,     # 사용자 값 2 -> 1일
        3: 2,     # 사용자 값 3 -> 2일
        4: 3,     # 사용자 값 4 -> 3일
        5: 4,     # 사용자 값 5 -> 4일
        6: 5,     # 사용자 값 6 -> 5일
        7: 6,     # 사용자 값 7 -> 6일
        8: 7,     # 사용자 값 8 -> 7일 (매일)
        99: None  # 모름/무응답
    }
    # user_data_original에서 'BE3_31' 값을 가져와 매핑하고, 매핑 규칙에 없으면 None 처리
    mapped_data['walking_freq'] = walk_freq_map.get(user_data_original.get('BE3_31'), None)
    # _____________________________________________________________________________________________________________________
    # BD1_11 (1년간 음주 빈도): 설문 코드를 실제 빈도에 가깝게 매핑 (연간 횟수 기준 - 다른 부서와 통일)
    alcohol_freq_map_annual = {
        1: 0,     # 사용자 값 1 -> 0 (최근 1년간 전혀 마시지 않았다)
        2: 6,     # 사용자 값 2 -> 6 (월1회 미만, 대략 연 6회)
        3: 12,    # 사용자 값 3 -> 12 (월1회정도, 연 12회)
        4: 42,    # 사용자 값 4 -> 42 (월2~4회, 월 평균 3.5회 * 12개월 = 42회)
        5: 130,   # 사용자 값 5 -> 130 (주2~3회, 주 평균 2.5회 * 52주 = 130회)
        6: 208,   # 사용자 값 6 -> 208 (주4회 이상, 주 평균 4회 * 52주 = 208회)
        8: 0,     # 사용자 값 8 -> 0 (마셔본적 없음)
        9: None   # 모름/무응답
    }
    # 'BD1_11' 값을 가져와 매핑하고, 매핑 규칙에 없으면 None 처리
    mapped_data['alcohol_freq_annual'] = alcohol_freq_map_annual.get(user_data_original.get('BD1_11'), None)
    # ___________________________________________________________________________________________________________________
    # L_BR_FQ (아침식사 빈도): 설문 코드를 주당 횟수 평균으로 매핑 (이건 원래 주당이었으므로 그대로 사용)
    breakfast_freq_map_weekly = {
        1: 6.0,   # 사용자 값 1 -> 6 (주 5~7회 (평균 6회))
        2: 3.5,   # 사용자 값 2 -> 3.5 (주 3~4회 (평균 3.5회))
        3: 1.5,   # 사용자 값 3 -> 1.5 (주 1~2회 (평균 1.5회))
        4: 0.0,   # 사용자 값 4 -> 0 (거의 안 먹음)
        9: None   # 모름/무응답
    }
    # 'L_BR_FQ' 값을 가져와 매핑하고, 매핑 규칙에 없으면 None 처리
    mapped_data['breakfast_freq'] = breakfast_freq_map_weekly.get(user_data_original.get('L_BR_FQ'), None)
    # ___________________________________________________________________________________________________________________
    # 추가: 아이콘 결정을 위한 "주간 음주 빈도" 파생 (연간 빈도에서 계산)
    # 이 부분은 BD1_11이 'alcohol_freq_annual'로 먼저 매핑된 후, 아이콘을 위해 다시 주간 단위로 변환
    if mapped_data.get('alcohol_freq_annual') is not None:
        if mapped_data['alcohol_freq_annual'] == 0:
            mapped_data['alcohol_freq_weekly'] = 0.0
        else:
            # 연간 횟수를 주간 횟수로 변환
            # (예: 6 (연 6회) -> 6/52 = 0.115)
            # (예: 12 (연 12회) -> 12/52 = 0.23)
            # (예: 42 (연 42회) -> 42/52 = 0.80)
            # (예: 130 (연 130회) -> 130/52 = 2.5)
            # (예: 208 (연 208회) -> 208/52 = 4.0)
            mapped_data['alcohol_freq_weekly'] = mapped_data['alcohol_freq_annual'] / 52.0
    else:
        mapped_data['alcohol_freq_weekly'] = None # 원본이 None이면 주간도 None
    return mapped_data
# Icon status determination function
def get_icon_status(mapped_data):
    # Retrieve mapped values, safely handling cases where keys might be missing or values are None
    # np.isnan은 float에만 작동하므로, None인 경우를 먼저 체크합니다.
    smoking_val = mapped_data.get('tobacco')
    drinking_val = mapped_data.get('alcohol_freq_weekly') # 주간 음주 빈도 사용
    breakfast_val = mapped_data.get('breakfast_freq')
    walking_val = mapped_data.get('walking_freq')
    return {
        # Smoking: 'on' if smoking amount is 1 or more, otherwise 'off'
        "smoking_icon": "on" if smoking_val is not None and smoking_val >= 1 else "off",
        # Drinking: 'on' if alcohol frequency is 2.0 or more per week, otherwise 'off'
        # (예: 2.5 times/week for category 5)
        "drinking_icon": "on" if drinking_val is not None and drinking_val >= 2.0 else "off",
        # Breakfast: 'on' if breakfast frequency is 2.0 or less per week, otherwise 'off'
        # (예: 1.5 times/week for category 3, 0.0 for category 4)
        "breakfast_icon": "on" if breakfast_val is not None and breakfast_val <= 2.0 else "off",
        # Walking: 'on' if walking frequency is 2 days/week or less, otherwise 'off'
        # (예: 0 days/week for category 1, 1 day/week for category 2)
        "walking_icon": "on" if walking_val is not None and walking_val <= 2 else "off"
    }
# --- Example Usage ---
# User input values (원본 설문 코드 기반의 키와 값 사용)
user_data = {
    "member_id": "MEM001",
    "checkup_id": "CHK20240525",
    "tobacco": 4,          # Tobacco amount (예: 개비 수)
    "BD1_11": 3,           # Alcohol frequency (예: 3: 월1회정도) - 원본 설문 코드 사용
    "L_BR_FQ": 2,          # Breakfast frequency (예: 2: 주3~4회) - 원본 설문 코드 사용
    "BE3_31": 1,           # Walking frequency (예: 1: 전혀 하지 않음) - 원본 설문 코드 사용
}
# Map the raw user input to standardized values
mapped_data = map_user_raw_input(user_data)
# Determine the icon status based on mapped data
icon_status = get_icon_status(mapped_data)
print(f"Mapped Data (연간/주간 통합): {mapped_data}")
print(f"Icon Status: {icon_status}")
# --- Database Connection and Insertion ---
# DB connection parameters
db_host = 'health-db.c1cwqmcao40y.ap-northeast-2.rds.amazonaws.com'
db_user = 'admin'
db_password = 'HealthCheckup123!'
db_name = 'health'
conn = None # Initialize conn to None
try:
    # Connect to the database
    conn = pymysql.connect(
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )
    with conn.cursor() as cursor:
        # SQL INSERT statement
        sql = """
        INSERT INTO icon_status (
            member_id, checkup_id, smoking_icon, drinking_icon, breakfast_icon, walking_icon
        ) VALUES (%s, %s, %s, %s, %s, %s)
        """
        # Execute the INSERT statement with mapped data and icon statuses
        cursor.execute(sql, (
            user_data["member_id"], # Using original member_id from user_data
            user_data["checkup_id"], # Using original checkup_id from user_data
            icon_status["smoking_icon"],
            icon_status["drinking_icon"],
            icon_status["breakfast_icon"],
            icon_status["walking_icon"]
        ))
    # Commit the transaction to save changes to the database
    conn.commit()
    print("데이터 저장 완료 (Data saved successfully)")
except pymysql.Error as e:
    # Handle database errors
    print(f"데이터베이스 오류 발생: {e}")
    if conn:
        conn.rollback() # Rollback changes if an error occurs
except Exception as e:
    # Handle any other unexpected errors
    print(f"예상치 못한 오류 발생: {e}")
finally:
    # Ensure the database connection is closed
    if conn:
        conn.close()
        print("데이터베이스 연결 종료 (Database connection closed)")

Mapped Data (연간/주간 통합): {'member_id': 'MEM001', 'checkup_id': 'CHK20240525', 'tobacco': 4, 'BD1_11': 3, 'L_BR_FQ': 2, 'BE3_31': 1, 'walking_freq': 0, 'alcohol_freq_annual': 12, 'breakfast_freq': 3.5, 'alcohol_freq_weekly': 0.23076923076923078}
Icon Status: {'smoking_icon': 'on', 'drinking_icon': 'off', 'breakfast_icon': 'off', 'walking_icon': 'on'}
데이터 저장 완료 (Data saved successfully)
데이터베이스 연결 종료 (Database connection closed)
